# 2.1.2 Algebraic data types

User-defined types in object-oriented languages are specified through class or trait declarations. Types can also be specified from already existing user-defined types through the inheritance mechanism.  

In functional programming, the rules for declaring new types are different: no inheritance or classes, just _products_, _sums_ and _exponentiation_ of types. Because of the correspondence with arithmetic (which goes beyond the terminology!), these types are called **algebraic data types** (ADTs). 

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 15. Case Classes and Pattern Matching

__[Scala book (online)](https://docs.scala-lang.org/overviews/scala-book/introduction.html)__.

- [Match Expressions](https://docs.scala-lang.org/overviews/scala-book/match-expressions.html)
- [Case classes](https://docs.scala-lang.org/overviews/scala-book/case-classes.html)
- [Case objects](https://docs.scala-lang.org/overviews/scala-book/case-objects.html)

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapters 19. Functional Programming as Algebra 

[__Tony Morris on ADTs__](https://about.chatroulette.com/posts/algebraic-data-types/)

## Product types

A value of product type $T_1 * T_2$ is created with a value of $T_1$ **and** a value of $T_2$. The constructor function is:
  - `create: (T1, T2) -> T1 * T2` (create is a 2-ary function). 

Given a value of a product type, we can obtain back both values with observers:
  - `fst: T1 * T2 -> T1` 
  - `snd: T1 * T2 -> T2`
  


### Products in Scala

In Scala, products are commonly called tuples. They are defined through classes that declare one public value field (conventionally named `_1`, `_2`, etc.) for each product member type.

In [ ]:
// type IntAndString = Int * String



Thus, the constructor of the product, `create`, is the constructor of the class, and the observers `fst` y `snd` the member fields `_1` y `_2`.

### Record types

Records are like products, but we can tag the member types with a given label. For instance:

In [ ]:
// type Rectangle = {width: Int * height: Int}



In [ ]:
// type Circle = {radius: Int}



In [ ]:
// type Triangle = {width: Int}



### Scala case classes

There are several things which are desirable to work with products/records:
 - Create new product objects without having to invoke `new`
 - Equality of product objects by value, not by reference
 - Off-the-shelf hash code
 - Pattern matching (more on this later on)


Create objects without `new`, is that possible? Yes, it is!

In [ ]:
class Rectangle(
    val width: Int,
    val height: Int)



In [ ]:
val r1: Rectangle = new Rectangle(5, 10)

Default equality is by-reference, not by-value:

In [ ]:
new Rectangle(5, 10) == new Rectangle(5, 10)

So, we must re-define the `equals` method (rules for overriding `equals` are more [complex](https://alvinalexander.com/scala/how-to-define-equals-hashcode-methods-in-scala-object-equality), though):

In [ ]:
class Rectangle(
    val width: Int,
    val height: Int){
    
    override def equals(p1: Any): Boolean = 
        ???
}

object Rectangle{
    def apply(width: Int, height: Int): Rectangle = 
        new Rectangle(width, height)
}

Now, it works as expected:

In [ ]:
// Rectangle(5, 10) == Rectangle(5, 10)

This is tedious and repetitive, i.e. boilerplate. Scala can make it for us automatically, using so-called `case classes`: 

In [ ]:
class Rectangle(width: Int, height: Int)
class Circle(radius: Int)
class Triangle(width: Int)

This declaration of the `Rectangle` record is essentially equivalent to what we did manually: the `case` keyword tells the Scala compiler to generate a companion object with an `apply` constructor; override the `equals` and `hashCode` methods, among other things.

In [ ]:
// Rectangle(1,1) == Rectangle(1,1)

In [ ]:
// Rectangle(1,1).hashCode

### Standard products in Scala: `TupleN` classes

The standard library of Scala has already defined for us generic case classes that represent the n-ary products (up to 22). The rough definition of `Tuple2` goes like this:

In [ ]:
object Std{
    class IntAndString(
        val _1: Int, 
        val _2: String)
    
    class IntAndBooleanAndChar(
        val _1: Int,
        val _2: String, 
        val _3: Char)
}

And Scala offers syntactic sugar, both for Tuple types and values. So, instead of writing something like this: 

we can write it as follows:

### Why are products called _algebraic_?

This is an example to illustrate the analogy between algebraic data types and arithmetic.

In [ ]:
// Number of values of Boolean type: 2

// Number of values of (Boolean, Boolean) type: 2 * 2 


In general, types may be regarded as sets of values. Then, the cardinal of $A * B$, for types $A$ and $B$ is: $|A * B| = |A| * |B|$.

If product types are analogous to number multiplication, then, is there any type which corresponds to the number 1, i.e. the neutral element of the multiplication? It has to be a type $1$ such that $A*1 \cong A \cong 1*A$, where the sign $\cong$ represents the _isomorphism_ of types, i.e. the types do not need to be equal but there should be a 1-1 mapping (a bijection) between the values of $A*1$ and $A$.  


Since the type $1$ has to comply with the identity rules, we have that $|A*1| = |A|$, but then $|A| * |1| = |A|$. So, $|1| = 1$, i.e. $1$ must be the type with just one value, i.e. the equivalent to the singleton set. We can create such a unit type as follows: 

The `sealed` keyword forbids the extension of class `One` outside its compilation unit, in this case the jupyter cell. Thus the following code does not compile:

So, the only instance of class `One` is `theonlyone`, i.e. the only object which is instantiated in its compilation unit.

But we don't have to create the unit type ourselves: it already exists in the Scala standard library, and it's called `Unit`, and its only value is `()`:

The isomorphism $Boolean * 1 \cong Boolean$ is witnessed by the following functions: 

which satisfy:
- `from(to(b))=b`, for all `b: Boolean`, and 
- `to(from(b))=b` for all `b: (Boolean, Unit)`.

Note that any function that returns a value of type `Unit` is completely useless from a purely functional perspective, since we already know in advance which is the (only possible) value that it returns: `()`. Therefore, if such a function makes sense is because it does something else than returning values: it must have some side effect, i.e. it has to be _impure_. This is also why we can say that `Unit` is the Scala equivalent to Java's `void`.

### Working with products

The goal is to translate algebraic rectangles, circles, etc., into SVG and be able to display these shapes in the notebook. We want to implement a function with the following signature:

In [ ]:
def displayCircle(c: Circle): Unit = ??? 

Note the return type: we are interested in the side effect of displaying something in the screen, so we disregard the returning value. 

We may go with the following implementation:

In [ ]:
import almond.interpreter.api.DisplayData
/*    
def displayCircle(c: Circle): Unit = 
   display(DisplayData(Map("text/html" -> 
        s"""<svg width="200" height="200">
              <circle cx="100" cy="100" r=${c.radius} fill="red"/>
            </svg>""")))
*/

In [ ]:
// displayCircle(Circle(100))

This works, but which are the problems of this implementation? Surely, this is the function we need, but it's not *modular* since there are at least two aspects that are intermingled in the code: 
- The computation of the SVG code (the pure part)
- The actual displaying of the HTML thunk (the impure part)

Lack of modularity in this case leads to the following problems:
- Later on, we will need to compute the SVG code of larger pictures, which are made from circles, rectangles, etc., and we will certainly need for that to translate circles to SVG. But with the current implementation, we won't be able to *reuse* anything (but by copy-pasting). 
- Another problem with this implementation is *testability*: we can't unit test the single part that generates the SVG code alone. We can only do _integration_ testing, i.e. checking the effect on the screen to see if what we get is right or not. 

So, we should better modularise our function as follows:

In [ ]:
type SVG = String
type HTML = String

// Module 1: toSVG

def circleToSVG(c: Circle): SVG = 
    ???

// Module 2: HtmlData
import almond.interpreter.api.DisplayData

def HtmlData(svg: SVG): DisplayData = 
    ???

// Our very same original function, but now in a better shape
def displayCircle(c: Circle): Unit = 
    ???
    


In [ ]:
// displayCircle(Circle(49))

## Sum types       

Besides multiplying types, we can also _sum_ types. Given types $A$ and $B$, the sum type $A + B$ represents **either** a value of type $A$ **or** a value of type $B$. Therefore, we have that 

$|A + B| = |A| + |B|$

For instance (the symbol $:=$ is used to give a name to a type):

- $MaybeInt := Int + 1$. A value of this type may be an integer; if it is not, then it is the unit value (a value that we use to signal that it is not an integer). So, $|MaybeInt| = |Int| + |1| = |Int| + 1$
- $EitherIntOrString := Int + String$. A value of this type is either an integer or a string, i.e. $|EitherIntOrString| = |Int| + |String|$
- $Shape := Circle + Rectangle + Triangle$. If we have a value of type $Shape$, then we have either a $Circle$, a $Triangle$ or a $Rectangle$. So, $|Shape| = |Circle| + |Triangle| + |Rectangle|$

We create and observe values of a sum type $A + B$ with the following functions: 
- Injection functions: 
  - `injA: A -> A + B`
  - `injB: B -> A + B`
- Match function:
  - `match: (A -> C) -> (B -> C) -> A + B -> C`
  
Note that the `match` function is a higher-order function. Basically, it says: if I know how to obtain a $C$ from $A$ (using function $A \rightarrow C$), and I know how to obtain a $C$ from $B$, then I know how to obtain a $C$ from $A+B$ (since $A+B$ is either an $A$ or a $B$).

### Sum types in Scala

How do we define sum types in an object-oriented language like Scala? Basically, we need _inheritance_, but with a special twist: it has to be _sealed_.

In [ ]:
// type Shape = Rectangle + Triangle + Circle 



As we already saw, the keyword `sealed` prevents the extension of the inheritance hierarchy with new subclasses. This guarantees that the sum type will remain consistent everywhere, i.e. that whenever we have an instance of `Shape` it will be either a rectangle, a circle or a triangle, and nothing else.

We create values of type `Shape` by using the constructors of its subclasses:

And we _observe_ these values with _pattern matching_, as follows:

Each `case` declaration represents a function from the corresponding type to the common target result. Being `sealed`, the compiler can check whether some pattern matching expression is complete or not, and warn us in case it's not.

In [ ]:
// This is Almond specific. 
// Also, note that it's possible that this diretive does not work for some kernel versions
interp.configureCompiler(_.settings.nowarn.value = false)

In [ ]:
// A warning should be raised here
/*
val s: String = s1 match {
    // Rectangle => String
    case r: Rectangle => "Rectangle" : String
    // Triangle => String
    case t: Triangle  => "Triangle"  : String 
    // Circle => String
    case c: Circle    => "Circle"    : String
}
*/

### Standard sum types in Scala

The standard library of Scala provides two important sum types: [`Option`](https://www.scala-lang.org/api/current/scala/Option.html) and [`Either`](https://www.scala-lang.org/api/current/scala/util/Either.html). They can be defined as follows: 

In [55]:
object StdSumTypes{
    // Option
    
    // Either
    
    sealed abstract class Either[A, B]
    case class Left[A, B](value: A) extends Either[A, B]
    case class Right[A, B](b: B) extends Either[A, B]
    
    val a: Either[Int, Boolean] = Left(1)
    val b: Either[Int, Boolean] = Right(true)
    def c: Either[Int, Boolean] = ??? // new Either[Int, Boolean]
    
    
    a match {
        case l@Left(a: Int) => ??? 
        case r@Right(b: Boolean) => ???
    }
    
    a match {
        case l: Left[Int, Boolean] => ??? 
        case r: Right[Int, Boolean] => ???
    }
}

defined object StdSumTypes

These types are important for error handling. We will see how they allow us to get rid of exceptions, at least in the part of our code that we wish to be purely functional. Here it's a small example:

In [ ]:
// Using exceptions

def divideWithExceptions(a: Double, b: Double): Double =
    if (b==0) throw new Exception("divide by cero")
    else a/b

In [ ]:
// divideWithExceptions(5,0)

In [ ]:
// Using option

/*
def divideWithExceptions(a: Double, b: Double): Double =
    if (b==0) throw new Exception("divide by cero")
    else a/b
*/

We now return a value which indicates whether there was an error or not:

In [ ]:
// Using Either

def divideWithEither(a: Double, b: Double): Either[String, Double] =
    ???/*if (b==0) Option.empty[Double] // None
    else Some(a/b)*/

And now a value which, in case of error indicates the reason:

### The 0 type in Scala

If the unit type was the identity element for product types, is there any identity type for sums? It has to be a type which satisfies the following conditions: 

- $0 + A \cong A$
- $A + 0 \cong A$

But $|0 + A| = |0| + |A|$, so $|0| = 0$, i.e. the type $0$ can't be inhabited. In other words, it is a type such that there is no value of that type. We can define such a type in Scala as follows:

In [30]:
sealed abstract class Zero

defined class Zero

In [30]:
//val aValue = new Zero{}

Being sealed and having no instances declared in its cell, the class `Zero` is inhabited. 
But we don't have to defined it ourselves, since the identity element of sums in Scala is already defined in the Scala standard library: it's the type `Nothing`. Since we can't create instances of this type, the only thing that we can do if we have to return a value of this type, or assign a variable of this type a value, is to throw an exception:

In [ ]:
val u: Unit = ()

In [36]:
def v: Nothing = throw new Exception("no possible value")

defined function v

In [37]:
v

: 

The `???` expression in Scala means essentially an exception of type `Nothing`. Also, note that `Nothing` is the botton of the Scala inheritance hierarchy, i.e. `Nothing` is a subclass of any Scala class. That's why we can use `???` in place of any value in Scala.

In [38]:
def i: Int = ??? // throw new Exception("no value")

defined function i

In [39]:
i

: 

The isomorphism $Int + 0 \cong Int$ is witnessed by the following functions:

In [47]:
// IntOrNothing := Int + Nothing
def from(v: Int): Either[Int, Nothing] = 
    Left(v) : Either[Int, Nothing]

def to(v: Either[Int, Nothing]): Int = 
    v match {
        case l: Left[Int, Nothing] => l.value
        //case r: Right[Int, Nothing] => ???
    }

defined function from
defined function to

In [48]:
to(from(1)) == 1 // Para todo i: Int
to(from(3)) == 3 // Para todo i: Int
from(to(Left(1))) == Left(1) // Para todo v: Int + 0
from(to(Left(5))) == Left(5) // Para todo v: Int + 0

res47_0: Boolean = true
res47_1: Boolean = true
res47_2: Boolean = true
res47_3: Boolean = true

### More on pattern matching

Let's implement a function that calculates the area of a shape:

In [49]:
import scala.math._

sealed abstract class Shape
case class Rectangle(width: Int, height: Int) extends Shape
case class Circle(radius: Int) extends Shape
case class Triangle(base: Int) extends Shape


def area(shape: Shape): Double = 
    shape match {
        case r: Rectangle => 
            r.width * r.height : Double
        case c: Circle => 
            Pi * pow(c.radius,2) : Double
        case t: Triangle => 
            t.base * t.base / 2 : Double
    }

def area(rectangle: Rectangle): Double = 
    rectangle.width * rectangle.height
    

import scala.math._


defined class Shape
defined class Rectangle
defined class Circle
defined class Triangle
defined function area
defined function area

We can implement this function more conveniently, using extractors to _deconstruct_ the value and access more directly its member attributes: 

In [57]:
Rectangle.apply(1,2)
Rectangle(width = 1, height = 2)

val v: Rectangle = Rectangle(1,2)
val Rectangle(w,h) = v

res56_0: Rectangle = Rectangle(1, 2)
res56_1: Rectangle = Rectangle(1, 2)
v: Rectangle = Rectangle(1, 2)
w: Int = 1
h: Int = 2

In [54]:

def area(shape: Shape): Double = 
    shape match {
        case r@Rectangle(w, h) => 
            r.width * r.height // w * h
        case Circle(r) => Pi * pow(r, 2)
        case t@Triangle(b) => b * b / 2.0
    }


defined function area

This can also be used in `val` declarations:

In [ ]:
// val r: Rectangle = Rectangle(1,2)


We can also use _guards_ in `case` branches:

In [58]:

def bigShape(s: Shape): String = 
    s match {
        case Rectangle(w,h) => 
            if (w > 500 && h > 1000) "big rectangle"
            else "normal shape"
        
        case Circle(_) => 
            "normal shape"
        
        case Triangle(_) => 
            "normal shape"
    }


def bigShape2(s: Shape): String = 
    s match {
        case Rectangle(w,h) => 
            if (w > 500 && h > 1000) "big rectangle"
            else "normal shape"
        case _ => 
            "normal shape"
    }


def bigShape3(s: Shape): String = 
    s match {
        case Rectangle(w,h) if w > 500 && h > 1000 => 
            "big rectangle"
        case _ => 
            "normal shape"
    }


defined function bigShape
defined function bigShape2
defined function bigShape3

In [61]:
bigShape(Rectangle(501,10001))

res60: String = "big rectangle"

We can pattern match on specific _values_ of variables: 

In [71]:
{ val w = 1 ; { val w = 2; println(w) }}

2


w: Int = 1

In [72]:

def isRectangle(r: Shape, w: Int, h: Int): Boolean = 
    r match {
        case Rectangle(`w`, `h`) => true
        case _ => false
    }


defined function isRectangle

In [70]:
isRectangle(Rectangle(1,3), 1, 2)

res69: Boolean = false

We can pattern match repeatedly until several levels of nesting:

In [78]:
def foo(s: Either[Either[Int, String], 
                  (Int, Either[String, Boolean])]): Boolean = 
    s match {
        case Left(e: Either[Int, String]) =>
            e match {
                case Left(i: Int) => ???
                case Right(s: String) => ???
            }
        case Right(t: (Int, Either[String, Boolean])) =>
            t._2 match {
                case Left(s: String) => ???
                case Right(b: Boolean) => ???
            }
    }

defined function foo

This is actually more convenient than this way:

In [79]:
def foo(s: Either[Either[Int, String], 
                  (Int, Either[String, Boolean])]): Boolean = 
    s match {
        case Left(Left(i: Int)) => ??? 
        case Left(Right(s: String)) => ??? 
        case Right((i: Int, Left(s: String))) => ???
        case Right((i: Int, Right(b: Boolean))) => ???
    }

defined function foo

In [80]:
def foo(s: Either[Either[Int, String], 
                  (Int, Either[String, Boolean])]): Boolean = 
    s match {
        case Left(Left(i)) => ??? 
        case Left(Right(s)) => ??? 
        case Right((i, Left(s))) => ???
        case Right((i, Right(b))) => ???
    }

defined function foo

In [73]:
4*(5+2)==4*5+4*2

res72: Boolean = true

In [77]:
val v1: Either[Either[Int, String], (Int, Either[String, Boolean])] = 
    Left(Left(1 : Int): Either[Int, String])

val v2: Either[Either[Int, String], (Int, Either[String, Boolean])] = 
    Left(Right("hola": String))

val v3: Either[Either[Int, String], (Int, Either[String, Boolean])] = 
    Right(1, Left("hola"))

val v4: Either[Either[Int, String], (Int, Either[String, Boolean])] = 
    Right(1, Right(true))


v1: Either[Either[Int, String], (Int, Either[String, Boolean])] = Left(Left(1))
v2: Either[Either[Int, String], (Int, Either[String, Boolean])] = Left(
  Right("hola")
)
v3: Either[Either[Int, String], (Int, Either[String, Boolean])] = Right(
  (1, Left("hola"))
)
v4: Either[Either[Int, String], (Int, Either[String, Boolean])] = Right(
  (1, Right(true))
)

Note that `(Int+String)+Int*(String+Boolean)` $\cong$ `Int + String + Int*String + Int*Boolean`, i.e. four cases.


More details on pattern matching in Scala can be found [here](https://docs.scala-lang.org/tour/pattern-matching.html)

### Working with sum types

Similarly to what we did above with circles, we want now to display shapes: 

In [ ]:
sealed abstract class Shape
case class Rectangle(width: Int, height: Int) extends Shape
case class Triangle(width: Int) extends Shape
case class Circle(radius: Int) extends Shape

We already implemented a function to display circles. We need now similar functions to display rectangles and triangles:

In [ ]:
def circleToSVG(c: Circle): SVG = 
        s"""<svg width="200" height="200">
              <circle cx="100" cy="100" r=${c.radius} fill="red"/>
            </svg>"""

def triangleToSVG(c: Triangle): SVG = {
    val height = math.sqrt(3) * c.width / 2
    val x = 100-c.width/2
    val y = 100+height/2
    s"""<svg width="200" height="200">
      <polygon points="$x,$y ${x+(c.width/2)},${y-height} ${x+c.width},$y" fill="red" />
    </svg>"""
}

def rectangleToSVG(c: Rectangle): SVG = 
    s"""<svg width="200" height="200">
       <rect width=${c.width} height=${c.height} fill="red" />
    </svg>"""

The function `toSVG` that translates shapes to SVG looks as follows:

In [ ]:
/*def toSVG(s: Shape): SVG = 
    ???
    */

We can now define our function:

In [ ]:
/*def displayShape(s: Shape): Unit = 
    display(HtmlData(toSVG(s)))
    */

In [ ]:
// displayShape(Rectangle(200, 150))

In [ ]:
// displayShape(Triangle(200))

### Objects vs. ADTs

The object-oriented way for writing this code would be something as follows:

In [ ]:
abstract class Shape{
    def toSVG: SVG
    def toEPS: EPS
}

case class Rectangle(width: Int, height: Int) extends Shape{
    def toSVG: SVG = ???
}

case class Triangle(width: Int) extends Shape{
    def toSVG: SVG = ???
}

case class Circle(radius: Int) extends Shape{
    def toSVG: SVG = ???
}

Basically, objects encapsulate state and behaviour. In functional programming, state and behaviour are completely *decoupled*. Data values are pure data, whereas behaviour is implemented through functions that interpret those values. This is done this way in order to improve modularity and allow both concerns to evolve independently. For instance, we may later add an interpreter `toEPS`, and we would not need to modify the inheritance hierarchy (the class `Shape` could even be out of our codebase). 

## Exponent types

We already know how to build new types by _adding_ and _multiplying_ other types. We will see now that function types can be properly called _exponent_ types. Indeed, let's consider how many functions are there with type `Boolean => Boolean`:

In [83]:
// false -> ??? , true -> ??? 
//           2  *          2   = 2^2 = 4
def f0(b: Boolean): Boolean = 
    ???

// false -> false , true -> false 
def f1(b: Boolean): Boolean = 
    false

// false -> true , true -> true
def f2(b: Boolean): Boolean = 
    true

// false -> false , true -> true
def f3(b: Boolean): Boolean = 
    b

// false -> true , true -> false
def f4(b: Boolean): Boolean = 
    !b
/*
// false -> true , true -> false
def f4(b: Boolean): Boolean = 
    !b || true
*/

defined function f0
defined function f1
defined function f2
defined function f3
defined function f4

In [86]:
// |1 + Boolean| = |1| + |Boolean| = 1 + 2 = 3
val v1: Either[Unit, Boolean] = Left(())
val v2: Either[Unit, Boolean] = Right(false)
val v3: Either[Unit, Boolean] = Right(true)

v1: Either[Unit, Boolean] = Left(())
v2: Either[Unit, Boolean] = Right(false)
v3: Either[Unit, Boolean] = Right(true)

In [ ]:
// false -> ???, true -> ???
//           3    *        3  = 3^2 = 9
val f: Boolean => Either[Unit, Boolean] = 
    ???


In [87]:
// Left(()) -> ???, Right(false) -> ???, Right(true) -> ???
//              2        *           2       *           2 = 2^3 = 8        
val f: Either[Unit, Boolean] => Boolean = 
    ???


: 

In general, $|X\Rightarrow Y|=|Y|^{|X|}$, since for any $X$ we have $|Y|$ values available. But the correspondence with arithmetic exponents goes further, since the familiar laws of exponents:

$$ 
\begin{array}{rcl}
X^0 & = & 1 \\
1^X & = & 1 \\
X^1 & = & X \\
Z^{X+Y} & = &  Z^X * Z^Y \\
Z^{X*Y} & = & (Z^X)^Y \\
(Y*Z)^X & = & Y^X*Z^X \\
\end{array}
$$



can be recasted in terms of laws for algebraic data types as follows:

$$ 
\begin{array}{rcl}
0 \Rightarrow X & \cong & 1 \\
X \Rightarrow 1 & \cong & 1 \\
1 \Rightarrow X & \cong & X \\
(X+Y) \Rightarrow Z & \cong & (X \Rightarrow Z) * (Y \Rightarrow Z) \\
(X*Y) \Rightarrow Z & \cong & X \Rightarrow (Y \Rightarrow Z) \\
X \Rightarrow Y*Z & \cong & (X \Rightarrow Y)*(X \Rightarrow Z) \\
\end{array}
$$



And these isomorphisms hold! Let's see some examples.

## $X \Rightarrow 1 \cong 1$

Essentially, this isomorphism tells us that we only have one implementation of the function type $X \Rightarrow 1$, for any type $X$. Namely:

In [89]:
// -2^32 -> ..., 0 -> ..., 1 -> ..., 2 -> , .... , 2^32 -> ...
//           1     *    1     *  1   *      ....  *  1   = 1^|Int| = 1   
def f[X](i: X): Unit = 
   () 
    

defined function f

which agrees with the formula: $|X \Rightarrow 1| = |1|^{|X|} = 1$.

## $0 \Rightarrow X \cong 1$

The same happens for $0 \Rightarrow X$, for any type $X$: 

In [91]:
val x: Int = 1 : Int
val s: String = "" : String


x: Int = 1
s: String = ""

In [92]:
val x: Any = 1 : Int // Int <: Any
val s: Any = "" : String // String <: Any


x: Any = 1
s: Any = ""

In [94]:
def n: Nothing = ???
def v: Int = n // Nothing <: Int
def v2: String = n // Nothing <: String

defined function n
defined function v
defined function v2

In [95]:
def f[X](n: Nothing): X = 
    n : X

defined function f

(recall that `Nothing <: X` for any `X` in Scala). This agrees with the arithmetic formula $|0 => X|=|X|^0=1$.

## $(Y+Z) \Rightarrow X \cong (Y \Rightarrow X) * (Z \Rightarrow X)$

We show this isomorphism by implementing the following functions:

In [97]:
def from[X, Y, Z](f: Either[Y, Z] => X): (Y => X, Z => X) = 
    ??? : (Y => X, Z => X)

defined function from

In [97]:
def from[X, Y, Z](f: Either[Y, Z] => X): (Y => X, Z => X) = 
    (??? : Y => X, ??? : Z => X)

defined function from

In [99]:
def from[X, Y, Z](f: Either[Y, Z] => X): (Y => X, Z => X) = 
    ((y : Y) => ??? : X, (z : Z) => ??? : X)

defined function from

In [101]:
def from[X, Y, Z](f: Either[Y, Z] => X): (Y => X, Z => X) = 
    ((y : Y) => f(??? : Either[Y, Z]) : X, 
     (z : Z) => f(??? : Either[Y, Z]) : X)

defined function from

In [103]:
def from[X, Y, Z](f: Either[Y, Z] => X): (Y => X, Z => X) = 
    ((y : Y) => f(Left(y) : Either[Y, Z]) : X, 
     (z : Z) => f(Right(z): Either[Y, Z]) : X)

defined function from

In [104]:
def from[X, Y, Z](f: Either[Y, Z] => X): (Y => X, Z => X) = 
    (y => f(Left(y)), 
     z => f(Right(z)))

defined function from

In [105]:
def to[X, Y, Z](t: (Y => X, Z => X)): Either[Y, Z] => X = 
    ??? : (Either[Y, Z] => X)

defined function to

In [106]:
def to[X, Y, Z](t: (Y => X, Z => X)): Either[Y, Z] => X = 
    (e: Either[Y, Z]) => ??? : X

defined function to

In [107]:
def to[X, Y, Z](t: (Y => X, Z => X)): Either[Y, Z] => X = 
    (e: Either[Y, Z]) => e match {
        case Left(y: Y) => 
            ??? : X
        case Right(z: Z) => 
            ??? : X
    }

defined function to

In [107]:
def to[X, Y, Z](t: (Y => X, Z => X)): Either[Y, Z] => X = 
    (e: Either[Y, Z]) => e match {
        case Left(y: Y) => 
            t._1(y: Y) : X
        case Right(z: Z) => 
            t._2(z : Z) : X
    }

defined function to

The last pattern matching can also be written more concisely using so-called [partial functions](https://www.scala-lang.org/api/current/scala/PartialFunction.html): 

In [108]:
def to[X, Y, Z](t: (Y => X, Z => X)): Either[Y, Z] => X = 
    {
        case Left(y: Y) => 
            t._1(y: Y) : X
        case Right(z: Z) => 
            t._2(z : Z) : X
    }

defined function to

(see this [post](https://alvinalexander.com/scala/how-to-define-use-partial-functions-in-scala-syntax-examples/) for more information on Scala partial functions).

But we must also show, or at least _test_, that both functions are mutual inverses, i.e. that:

`from(to(f)) == f`, for all `f: (Y => X, Z => X)`

`to(from(f)) == f`, for all `f: Either[Y, Z] => X`

We will perform some unit testing here with the following two functions:

In [116]:
val ex1: Either[Boolean, Boolean] => Boolean = 
    _ => false

val ex2: (Boolean => Boolean, Boolean => Boolean) = 
    (_ => false, _ => true)


val ex3: (Boolean => Boolean, Boolean => Boolean) = 
    (b => b, b => !b)

ex1: Either[Boolean, Boolean] => Boolean = ammonite.$sess.cmd115$Helper$$Lambda$2796/1019884776@5781125c
ex2: (Boolean => Boolean, Boolean => Boolean) = (
  ammonite.$sess.cmd115$Helper$$Lambda$2797/309725107@3eb3cdac,
  ammonite.$sess.cmd115$Helper$$Lambda$2798/880744087@c028f4d
)
ex3: (Boolean => Boolean, Boolean => Boolean) = (
  ammonite.$sess.cmd115$Helper$$Lambda$2799/1314402051@66e18e38,
  ammonite.$sess.cmd115$Helper$$Lambda$2800/601370452@35ddfe4a
)

thus fixing types $X$, $Y$ and $Z$ to $Boolean$. Then, we need the following equality functions:

In [113]:
def equal2(f1: Boolean => Boolean, f2: Boolean => Boolean): Boolean = 
    f1(false) == f2(false) && 
    f1(true) == f2(true)

def equal1(t1: (Boolean => Boolean, Boolean => Boolean), 
          t2: (Boolean => Boolean, Boolean => Boolean)): Boolean = 
    equal2(t1._1, t2._1) && 
    equal2(t1._2, t2._2)

def equal3(f1: Either[Boolean, Boolean] => Boolean,
           f2: Either[Boolean, Boolean] => Boolean): Boolean = 
    f1(Left(false)) == f2(Left(false)) && 
    f1(Left(true)) == f2(Left(true)) && 
    f1(Right(false)) == f2(Right(false)) && 
    f1(Right(true)) == f2(Right(true))

defined function equal2
defined function equal1
defined function equal3

and, finally, we can perform our test: 

In [111]:
/*from(to(ex2))==ex2
to(from(ex1))==ex1
*/


In [117]:
equal3(to(from(ex1)), ex1) 
equal1(from(to(ex2)), ex2)
equal1(from(to(ex3)), ex3)

res116_0: Boolean = true
res116_1: Boolean = true
res116_2: Boolean = true